## Directives Processor:

This notebook explores generating and maintaining directives for the FAISS Indexes.

In [1]:
import json
import os
import pprint

In [2]:
# Functions for loading and saving data
def load_topics_data():
    topics_filename = 'topics_data.json'
    if os.path.exists(topics_filename):
        with open(topics_filename, "r") as file:
            topics_data = json.load(file)
    else:
        topics_data = {"topics": {}}
        save_topics_data(topics_data)
    
    return topics_data

def save_topics_data(topics_data):
    with open("topics_data.json", "w") as file:
        json.dump(topics_data, file, indent=4)

In [3]:
# Define the global variable (or reload it)
topics_data = load_topics_data()

In [4]:
def find_next_available_number(ids):
    # Convert string array elements to integers
    int_arr = [int(num) for num in ids]
    
    # Sort the integer array
    int_arr.sort()

    for i in range(len(int_arr)):
        if int_arr[i] != i + 1:
            return i + 1

    return len(int_arr) + 1

# This method looks for both the id and name so that it can be used to 
# find the topic's info and to also check to see if it already exists.
def get_topic(topic):
    # Search for the ID first
    _topic = topics_data["topics"].get(topic)
    
    if _topic:
        return _topic
    
    for _, _topic in topics_data["topics"].items():
        print('_topic', _topic)
        print('_topic.get("topic_title")', _topic.get("topic_title"))
        topic_name = _topic.get("topic_title", "").lower()
        if topic_name == topic.lower():
            return _topic
    
    return None

# In order to simplify the process, this method is only attempting to find an existing directive.
# The original directive ID is sufficient to modify the value
def get_directive_by_id(directive_id, include_topic=False):
#     global topics_data
    # First, get the topic from the directive ID
    split_id = directive_id.split('.')
    topic_id = split_id[0]
    topic = get_topic(topic_id)
    # The topic needs to already exist so that there is an id and name that has already been set
    if topic:
        # Now search for the directive ID
        _directive = topic.get('directives', {}).get(split_id[1])
        if include_topic:
            return topic, _directive
        return _directive
    
    if include_topic:
        return None, None
    return None

# This returns the directive_id and its directive
def get_directive_by_name(topic_id, directive_name):
    topic = get_topic(topic_id)
    if topic:
        for directive_id, directive in topic.get('directives', {}).items():
            directive_title = directive.get('directive_title', '')
            if directive_title.lower() == directive_name.lower():
                return directive_id, directive
            
    return None, None
            

# In order to simplify the process, this method is only attempting to find an existing subdirective.
def get_subdirective(directive_id, name_or_id):
#     global topics_data
    # First, get the directive from the subdirective ID.
    directive = get_directive_by_id(directive_id)
    if not directive:
        directive = get_directive_by_name(directive_id)
    
    if directive:
        subdirective = directive.get('subdirectives', {}).get(name_or_id)
        
        if subdirective:
            # In this case the name_or_id variable is the ID
            return name_or_id, subdirective
        
        # If you didn't find it by ID, search by name
        for subdirective_id, subdirective in directive.get('subdirectives', {}).items():
            # subdirective should just be a string so we can do a direct comparison at this point
            if name_or_id.lower() == subdirective.lower():
                return subdirective_id, subdirective
            
    return None, None

def add_topic(topic_id, topic):
    # Check to see if it already exists
    _topic = get_topic(topic_id)
    if not topic_id or not topic:
        raise Exception('A topic requires a topic ID and description')
    
    if _topic:
        # Don't process a second time if the ID exists
        print("Topic ID " + topic_id + " already exists.")
        return
    _topic = get_topic(topic)
    if _topic:
        # Same goes for the name
        print("Topic name " + topic + " already exists.")
        return
    
    # It's OK to create it
    topics_data['topics'][topic_id] = {'topic_title': topic, 'directives': {}}
    save_topics_data(topics_data)

def add_directive(topic_id, directive_title, directive_id=None):
    # Load the topic first
    _topic = get_topic(topic_id)
    if not _topic:
        print("Topic ID " + topic_id + " not found.")
        return
    
    # Make sure that the directive doesn't already exist
    _directive_id, _directive = get_directive_by_name(topic_id, directive_title)
#     print('DIRECTIVE:')
#     pprint.pprint(_directive)
    if _directive:
        # Don't process a second time if the directive exists
        print("Directive title " + directive_title + " already exists.")
        return _directive_id
    
    # If you are manually setting the directive_id, make sure that doesn't exist either
    if directive_id:
        _directive = get_directive_by_id(topic_id + '.' + directive_id)
        if _directive:
            # Don't process a second time if the directive ID exists
            print("Directive ID " + directive_id + " already exists.")
            return directive_id
    else:
        directive_id = find_next_available_number(_topic['directives'].keys())
    
    # Hasn't been found so you can create it and save
    # Add to the current topic
#     print('Updating topic:')
#     pprint.pprint(_topic, compact=True)
    _topic['directives'][str(directive_id)] = {'directive_title': directive_title, 'subdirectives': {}}
    
#     print('New topic:')
#     pprint.pprint(_topic, compact=True)
    # Update the topic and save
    topics_data['topics'][topic_id] = _topic
#     print('New TOPICS DATA:')
#     pprint.pprint(topics_data)
    save_topics_data(topics_data)
    # For convenience, return the new ID
    return directive_id
    
def add_subdirective(directive_id, subdirective_title, subdirective_id=None):
    # directive_id must be of the form <topic_id.directive_id>
    _subdirective_id, _subdirective_title = get_subdirective(directive_id, subdirective_title)
    if _subdirective_id:
        print("Subdirective ID " + _subdirective_id + "(" + _subdirective_title + ") already exists.") 
        return _subdirective_id
    
    # You will need the topic and directive in order to add a subdirective
    _topic, _directive = get_directive_by_id(directive_id, True)
    
    
    # If you are manually setting the directive_id, make sure that doesn't exist either
    if subdirective_id:
        _subdirective_id, _subdirective = get_subdirective(directive_id, subdirective_id)
        if _subdirective:
            # Don't process a second time if the directive ID exists
            print("Subdirective ID " + _subdirective_id + " already exists.")
            return subdirective_id
    else:
#         directive_id = find_next_available_number(_topic['directives'].keys())
        # Now that you have the directive, get the next available number in the list of subdirectives
        _subdirective_id = find_next_available_number(_directive['subdirectives'].keys())
        
    # Add the new subdirective to the directive
    print('Saving subdirective with ID', _subdirective_id)
    _directive['subdirectives'][str(_subdirective_id)] = subdirective_title
    # Update the topic
    id_split = directive_id.split('.')
    topic_id = id_split[0]
    _directive_id = id_split[1]
    _topic['directives'][str(_directive_id)] = _directive
    # Update the topic and save
    topics_data['topics'][topic_id] = _topic
    save_topics_data(topics_data)  
    # For convenience, return the new ID
    return _subdirective_id

In [5]:
add_topic('AI', 'Artificial Intelligence and Machine Learning')

directive_id = add_directive("AI", "Global Implementation of AI and ML")
add_subdirective("AI."+str(directive_id), "Identify key AI/ML initiatives in various countries.")
add_subdirective("AI."+str(directive_id), "Analyze adoption rates and trends of AI/ML technologies.")
add_subdirective("AI."+str(directive_id), "Compare and contrast scope and scale of AI/ML applications in different nations.")

directive_id = add_directive("AI", "Policy and Regulation Variance")
add_subdirective("AI." + str(directive_id), "Investigate legal frameworks and policies governing AI/ML in different countries.")
add_subdirective("AI." + str(directive_id), "Assess ethical considerations and regulatory disparities.")
add_subdirective("AI." + str(directive_id), "Highlight geopolitical implications of AI/ML disparities among nations.")

directive_id = add_directive("AI", "Technological Advancements and Competitiveness")
add_subdirective("AI." + str(directive_id), "Evaluate advancements in AI/ML research and development worldwide.")
add_subdirective("AI." + str(directive_id), "Examine competitive edge of different nations in AI/ML innovation.")
add_subdirective("AI." + str(directive_id), "Compare investments made by various countries in AI/ML infrastructure and education.")


Topic ID AI already exists.
Directive title Global Implementation of AI and ML already exists.
Subdirective ID 1(Identify key AI/ML initiatives in various countries.) already exists.
Subdirective ID 2(Analyze adoption rates and trends of AI/ML technologies.) already exists.
Subdirective ID 3(Compare and contrast scope and scale of AI/ML applications in different nations.) already exists.
Directive title Policy and Regulation Variance already exists.
Subdirective ID 1(Investigate legal frameworks and policies governing AI/ML in different countries.) already exists.
Subdirective ID 2(Assess ethical considerations and regulatory disparities.) already exists.
Subdirective ID 3(Highlight geopolitical implications of AI/ML disparities among nations.) already exists.
Directive title Technological Advancements and Competitiveness already exists.
Subdirective ID 1(Evaluate advancements in AI/ML research and development worldwide.) already exists.
Subdirective ID 2(Examine competitive edge of di

'3'

In [14]:
file_path = 'topics_and_directives.txt'
with open(file_path, 'r') as file:
    current_topic_id = None
    current_directive_id = None
    for line in file:
        print('Processing line:\n\t', line)
        line = line.rstrip('\n')  # Remove newline character
        
        # Count leading tabs for indentation level
        indent_level = 0
        while line.startswith('\t'):
            indent_level += 1
            line = line[1:]  # Remove one tab character from the beginning
        
        line = line.strip()  # Remove extra spaces
        print('Indent level:', indent_level)
        print('Processing line:\n\t', line)
        
        if indent_level == 0:
            # Adding a new topic
            topic_id, topic_name = line.split(':')
            print('Adding topic and topic id', topic_name.strip(), topic_id.strip())
            add_topic(topic_id.strip(), topic_name.strip())
            current_topic_id = topic_id.strip()
            current_directive_id = None
        elif indent_level == 1:
            # Adding a new directive under the current topic
            directive_id = add_directive(current_topic_id, line)
            current_directive_id = directive_id
        elif indent_level == 2 and current_directive_id:
            # Adding a new sub-directive under the current directive
            add_subdirective(str(current_topic_id) + '.' + str(current_directive_id), line)

Processing line:
	 AI:Artificial Intelligence and Machine Learning

Indent level: 0
Processing line:
	 AI:Artificial Intelligence and Machine Learning
Adding topic and topic id Artificial Intelligence and Machine Learning AI
Topic ID AI already exists.
Processing line:
	 	Global Implementation of AI and ML

Indent level: 1
Processing line:
	 Global Implementation of AI and ML
Directive title Global Implementation of AI and ML already exists.
Processing line:
	 		Identify key AI/ML initiatives in various countries.

Indent level: 2
Processing line:
	 Identify key AI/ML initiatives in various countries.
Subdirective ID 1(Identify key AI/ML initiatives in various countries.) already exists.
Processing line:
	 		Analyze adoption rates and trends of AI/ML technologies.

Indent level: 2
Processing line:
	 Analyze adoption rates and trends of AI/ML technologies.
Subdirective ID 2(Analyze adoption rates and trends of AI/ML technologies.) already exists.
Processing line:
	 		Compare and contras

In [13]:
def get_directive_text_from_directive(directive_data):
#     print('The directive data is: ', directive_data)
    directive_title = directive_data.get('directive_title')
    subdirectives = directive_data.get('subdirectives')
#     print('The subdirective data is: ', subdirectives)
    text = f"\t{directive_title}\n"
    for key, value in subdirectives.items():
        text += f"\t\t{value}\n"
        
    return text    

def get_directive_text(topic_id, directive_id):
    directive_data = get_directive_by_id(topic_id + '.' + directive_id)
    return get_directive_text_from_directive(directive_data)

def get_topic_text(topic_id):
    _topic = get_topic(topic_id)
    topic_title = _topic.get('topic_title')
    directives = _topic.get('directives', {})
    text = f"{topic_title}\n"
    
    for key, directive_data in directives.items():
#     for directive_data in directives:
        text += get_directive_text_from_directive(directive_data)

    return text

def get_topic_titles():
    titles = []
    for _, _topic in topics_data["topics"].items():
        titles.append(_topic.get("topic_title"))
    return titles

print('TOPIC TITLES:', get_topic_titles())
print('\n\nAI TOPIC TEXT:', get_topic_text('AI'))
print('\n\nTHE DIRECTIVE TEXT IS:\n', get_directive_text('AI', '2'))

TOPIC TITLES: ['Artificial Intelligence and Machine Learning', 'Documents covering climate change, global warming, environmental impacts, renewable energy, etc.']


AI TOPIC TEXT: Artificial Intelligence and Machine Learning
	Global Implementation of AI and ML
		Identify key AI/ML initiatives in various countries.
		Analyze adoption rates and trends of AI/ML technologies.
		Compare and contrast scope and scale of AI/ML applications in different nations.
	Policy and Regulation Variance
		Investigate legal frameworks and policies governing AI/ML in different countries.
		Assess ethical considerations and regulatory disparities.
		Highlight geopolitical implications of AI/ML disparities among nations.
	Technological Advancements and Competitiveness
		Evaluate advancements in AI/ML research and development worldwide.
		Examine competitive edge of different nations in AI/ML innovation.
		Compare investments made by various countries in AI/ML infrastructure and education.



THE DIRECTIVE TE

In [8]:
# directive_id = add_directive("AI", "Global Implementation of AI and ML")
# add_subdirective("AI."+str(directive_id), "Identify key AI/ML initiatives in various countries.")
# add_subdirective("AI."+str(directive_id), "Analyze adoption rates and trends of AI/ML technologies.")
# add_subdirective("AI."+str(directive_id), "Compare and contrast scope and scale of AI/ML applications in different nations.")
# directive_id = add_directive("AI", "Policy and Regulation Variance")
# directive_id = add_directive("AI", "Technological Advancements and Competitiveness")
# add_subdirective("AI."+str(directive_id), "Evaluate advancements in AI/ML research and development worldwide.")
# add_subdirective("AI."+str(directive_id), "Examine competitive edge of different nations in AI/ML innovation.")
# add_subdirective("AI."+str(directive_id), "Compare investments made by various countries in AI/ML infrastructure and education.")
